In [1]:
import pandas as pd
from ast import literal_eval

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows

In [3]:
cd D:\Project\Toolkit_for_Preprocessing_MXH\ViHOS_chunking

D:\Project\Toolkit_for_Preprocessing_MXH\ViHOS_chunking


c:\Users\ndp17\.pyenv\pyenv-win\versions\3.10.5\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Warm up

In [16]:
train_path = r"Data\Raw_data\train.csv"
dev_path = r"Data\Raw_data\dev.csv"
test_path = r"Data\Raw_data\test.csv"

In [17]:
train = pd.read_csv(train_path)
dev = pd.read_csv(dev_path)
test = pd.read_csv(test_path)
test['index_spans'] = test['index_spans'].apply(literal_eval)
train['index_spans'] = train['index_spans'].apply(literal_eval)
dev['index_spans'] = dev['index_spans'].apply(literal_eval)

headers = ['Unnamed: 0',  'content', 'index_spans']
train.columns = headers
dev.columns = headers
test.columns = headers
test.head(2)

,Unnamed: 0,content,index_spans
0,0,Anh bar .,[]
1,1,Hello thầy,[]


# Pre-processing

In [18]:
from Code.Preprocessing import unicode
# Apply the replacement function to the 'content' column
test['content'] = test['content'].apply(unicode)
train['content'] = train['content'].apply(unicode)
dev['content'] = dev['content'].apply(unicode)


In [19]:
import os
folder_path = "spans_text"
if not os.path.exists(folder_path):
    # Create the folder if it doesn't exist
    os.makedirs(folder_path)

In [20]:
df = train[['index_spans', 'content']]
df.columns = ['spans', 'text']
df.to_csv(r'spans_text\df_train.csv')

df = dev[['index_spans', 'content']]
df.columns = ['spans', 'text']
df.to_csv(r'spans_text\df_dev.csv')

df = test[['index_spans', 'content']]
df.columns = ['spans', 'text']
df.to_csv(r'spans_text\df_test.csv')

# Tokeniner

In [21]:
# from vncorenlp import VnCoreNLP
# annotator = VnCoreNLP(r"D:\Project\Toolkit_for_Preprocessing_MXH\ViHOS\Code\VnCoreNLP", annotators="wseg", max_heap_size='-Xmx500m')

from vncorenlp import VnCoreNLP

# Ensure that the JAR file path is correct
annotator = VnCoreNLP(r"D:\\Project\\Toolkit_for_Preprocessing_MXH\\ViHOS\\Code\\VnCoreNLP\\VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')


In [22]:
text = test['content'][325]
annotator_text = annotator.tokenize(text)
tokens = []
for i in range(len(annotator_text)):
  for j in range(len(annotator_text[i])):
    tokens.append(annotator_text[i][j])
tokens


['A', 'méo', 'nên', 'về', 'nước', 'thì', 'mới', 'đúng']

In [23]:
def tokenize_word(text, pos):
    tokens = [token for sentence in annotator.tokenize(text) for token in sentence]
    alignment, start = [], 0

    for t in tokens:
        if t == "_":
            res = text.find(t, start)
        else:
            t = t.lstrip("_").replace("_", " ")
            res = text.find(t, start)

        alignment.append(pos[res:res + len(t)])
        start = res + len(t)

    assert len(tokens) == len(alignment)
    return tokens, alignment

def annotate(spans, alignment, tokens):
    annotations = pd.DataFrame({'Tokens': tokens, 'Tag': ['O'] * len(tokens)})

    for span in spans:
        for i, align in enumerate(alignment):
            if align[-1] < span[0]:
                continue
            elif align[0] <= span[0] <= align[-1]:
                annotations.at[i, 'Tag'] = 'B-T'
            elif span[0] < align[0] <= span[-1]:
                annotations.at[i, 'Tag'] = 'I-T'
            elif align[0] > span[-1]:
                break

    return annotations['Tag']


In [ ]:
from Code.Preprocessing import load_data

In [24]:
test_data = load_data(r'spans_text\df_test.csv')
train_data = load_data(r'spans_text\df_train.csv')
dev_data = load_data(r'spans_text\df_dev.csv')
test_data[30]

{'text': 'đ m, thầy giáo cũng sống ảo',
 'spans': [[0, 2]],
 'text_spans': ['đ m']}

# Annotate sentence-based Chunking

In [4]:
train_path = ("Data\spans_text\df_train.csv")
dev_path = ("Data\spans_text\df_dev.csv")
test_path = ("Data\spans_text\df_test.csv")

In [5]:
from Code.Preprocessing import load_data
train_data = load_data(train_path)
dev_data = load_data(dev_path)
test_data = load_data(test_path)
test_data[128]

{'text': 'Không thể tưởng tượng nổi... Bọn chúng ăn bất chấp thủ đoạn',
 'spans': [[29, 37], [42, 58]],
 'text_spans': ['Bọn chúng', 'bất chấp thủ đoạn']}

In [18]:
from Code.Preprocessing import dupplicate_punctuation, sentence_based_chunking
import numpy as np
import pandas as pd

def data_chunking(data):
    formated_data = []


    for d in data:
        text, spans, text_spans = d['text'], d['spans'], d['text_spans']
        pos = list(range(len(text)))  # Create a position list
        text, pos, spans = dupplicate_punctuation(text, pos, spans)  # Clean up punctuation
        annotations = sentence_based_chunking(text, pos, spans)
        annotations = [annotation.values() for annotation in annotations]

        # Combine tokens and their corresponding annotations
        formated_data.extend(annotations)  # Using zip for better performance
        formated_data.append((None, None, None))  # Append a marker for sentence separation


    # Create a DataFrame from the formatted data
    df_final = pd.DataFrame(formated_data, columns=['Chunk', 'Tag', 'Spans'])

    # Generate sentence IDs
    sentence_id = []
    sentence = 0
    for word in df_final['Chunk']:
        if word is not None:
            sentence_id.append(sentence)
        else:
            sentence_id.append(np.nan)
            sentence += 1

    df_final['sentence_id'] = sentence_id
    df_final.dropna(inplace=True)  # Remove rows where Word is None
    df_final['sentence_id'] = df_final['sentence_id'].astype("int64")  # Convert to int64
    df_final = df_final[['Chunk', 'Tag', 'sentence_id', 'Spans']]

    return df_final

In [25]:
test = data_chunking(test_data)
train = data_chunking(train_data)
dev = data_chunking(dev_data)
test.head()

,Chunk,Tag,sentence_id,Spans
0,Anh bar .,0.0,0,[]
2,Hello thầy,0.0,1,[]
4,Từ lúc mấy bro cmt cực kì cl gì đấy là rất khó để ngó được cái cmt hay buồn thực sự 🙁,1.0,2,"[[26, 27]]"
6,Nghe trần dần thấy cười đau cả bụng,0.0,3,[]
8,Idol tui có khác.,0.0,4,[]


In [27]:
train.reset_index(inplace=True)
dev.reset_index(inplace=True)
test.reset_index(inplace=True)

In [29]:
import os
folder_path = "Data\Chunking_data"
if not os.path.exists(folder_path):
    # Create the folder if it doesn't exist
    os.makedirs(folder_path)

train.to_csv(r'Data\Chunking_data\train.csv', index=False)
dev.to_csv(r'Data\Chunking_data\dev.csv', index=False)
test.to_csv(r'Data\Chunking_data\test.csv', index=False)